In [15]:
import numpy as np
import time
import numba
from numba import jit
from SGHMC.sghmc import sghmc, minibatch_data

In [2]:
@jit
def minibatch_data_numba(data, batch_size, random_seed=45):
    """
    Create minibatch samples from the dataset
    """
    n = data.shape[0]
    p = data.shape[1]
    if n % batch_size != 0:
        n = (n // batch_size) * batch_size
    ind = np.arange(n)
    np.random.shuffle(ind)
    n_minibatches = n // batch_size
    data = data[ind].reshape(batch_size, p, n_minibatches)
    return(data, n_minibatches)

In [3]:
@jit
def sghmc_numba(gradU, eps, C, inv_M, theta_0, V_hat, data, batch_size, burn_in, n_iter=500):
    '''
    Define SGHMC as described in the paper Stochastic Gradient Hamilton Monte Carlo, 
    ICML, Beijing, China, 2014 by
    Tianqi Chen, Emily B. Fox, Carlos Guestrin.

    The inputs are:
    gradU = gradient of U
    eps = the learning rate
    C = user specified friction term
    inv_M = inverse of the mass matrix
    theta_0 = initial value of parameter sampling
    V_hat = estimated covariance matrix using empirical Fisher information
    batch_size = size of a minibatch in an iteration
    burn_in = number of iterations to drop
    n_iter = number of samples to generate

    The outpit is:
    theta_samples: a np.array of positions of theta.
    '''

    # parameter vector dimension
    p = theta_0.shape[0]
    # number of samples
    n = data.shape[0]
    # placeholder for theta samples
    theta_samples = np.zeros((p, n_iter))
    theta_samples[:, 0] = theta_0
    
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = (V_hat * eps) / 2
    Sigma = np.linalg.cholesky(2 * (C - beta_hat) * eps)
    
    # data
    mini_data, n_batches = minibatch_data_numba(data, batch_size)

    # assert batch size to be <= the amount of data
    if (batch_size > data.shape[0]): 
        print("Error: batch_size cannot be bigger than the number of samples")
        return
    
    # loop through algorithm to get n iteration samples
    for i in range(n_iter - 1):
        theta = theta_samples[:, i]
        # resample momentum at each new iteration
        M = np.linalg.cholesky(np.linalg.inv(inv_M))
        momen = M@np.random.randn(p).reshape(p, -1)
        
        # sghmc sampler
        for j in range(n_batches):
            theta = theta + (eps*inv_M@momen).flatten()
            gradU_batch = gradU(theta, mini_data[:,:,j], n, batch_size).reshape(p, -1)
            momen = momen - eps*gradU_batch - eps*C@inv_M@momen + Sigma@np.random.randn(p).reshape(p, -1)
            
        theta_samples[:, i+1] = theta
        
    return theta_samples[:, burn_in:]

In [9]:
@jit
def sghmc_numba2(gradU, eps, C, inv_M, theta_0, V_hat, data, batch_size, burn_in, n_iter=500):
    '''
    Define SGHMC as described in the paper

    The inputs are:
    gradU = gradient of U
    eps = eps^2 M^(-1)
    C = 
    inv_M = 
    theta_0 = initial value of parameter sampling
    V_hat = 
    n_iter = number of samples to generate

    The outpit is:
    A np.array of positions of theta.
    '''

    # parameter vector dimension
    p = theta_0.shape[0]
    # number of samples
    n = data.shape[0]
    # placeholder for theta samples
    theta_samples = np.zeros((p, n_iter))
    theta_samples[:, 0] = theta_0
    
    # fix beta_hat as described on pg. 6 of paper
    beta_hat = (V_hat * eps) / 2
    Sigma = np.linalg.cholesky(2 * (C - beta_hat) * eps)
    
    # data
    mini_data, n_batches = minibatch_data(data, batch_size)

    # assert batch size to be <= the amount of data
    if (batch_size > data.shape[0]): 
        print("Error: batch_size cannot be bigger than the number of samples")
        return
    
    # loop through algorithm to get n iteration samples
    for i in range(n_iter - 1):
        theta = theta_samples[:, i]
        # resample momentum at each new iteration
        M = np.linalg.cholesky(np.linalg.inv(inv_M))
        momen = M@np.random.randn(p).reshape(p, -1)
        
        # sghmc sampler
        for j in range(n_batches):
            theta = theta + (eps*inv_M@momen).flatten()
            gradU_batch = gradU(theta, mini_data[:,:,j], n, batch_size).reshape(p, -1)
            momen = momen - eps*gradU_batch - eps*C@inv_M@momen + Sigma@np.random.randn(p).reshape(p, -1)
            
        theta_samples[:, i+1] = theta
        
    return theta_samples[:, burn_in:]

## Example 1

In [4]:
np.random.seed(663)
n = 200
x = np.zeros((200, 1))
theta_0 = np.array([0])
p = theta_0.shape[0]
eps = 0.1
C = np.eye(1)
V_hat = np.eye(1)*4
batch_size = 5
n_iter = 200
inv_M = np.eye(p)
burn_in = 50

def noisy_gradU(theta, x, n, batch_size):
    '''noisy gradient from paper fig1'''
    return -4*theta + 4*theta**3 + np.random.normal(0,2)

In [7]:
%timeit sghmc(noisy_gradU, eps, C, inv_M, theta_0, V_hat, x, batch_size, burn_in, n_iter) #no jit

180 ms ± 7.97 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [20]:
%timeit sghmc_numba(noisy_gradU, eps, C, inv_M, theta_0, V_hat, x, batch_size, burn_in, n_iter) #jit to both

188 ms ± 23.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [19]:
%timeit sghmc_numba2(noisy_gradU, eps, C, inv_M, theta_0, V_hat, x, batch_size, burn_in, n_iter) #jit to sghmc

178 ms ± 5.29 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


## Example 2

In [22]:
from autograd import jacobian
import autograd.numpy as np
# compute log(theta)
def log_prior(theta):
    return (-1/(2*10))*theta.T@theta

# compute log(x\theta)
def log_like(theta, x):
    return np.log(0.5 * np.exp(-0.5*(theta[0]-x)**2) + 0.5* np.exp(-0.5*(theta[1]-x)**2))

# function of U(theta)
def U(theta, x, n, batch_size):
    return -log_prior(theta) - (n/batch_size)*sum(log_like(theta, x))

gradU = jacobian(U, argnum = 0)

np.random.seed(663)
n = 200
theta_0 = np.array([0,0])
p = theta_0.shape[0]
theta = np.array([-3,3]).reshape(p,1)
x = np.r_[np.random.normal(theta[0], 1, (n,1)),
              np.random.normal(theta[1], 1, (n,1))].reshape(-1,1)
eps = 0.01
C = np.eye(p)
V_hat = np.eye(p)
batch_size = 80
n_iter = 200
inv_M = np.eye(p)
burn_in = 50

In [23]:
%timeit sghmc(gradU, eps, C, inv_M, theta_0, V_hat, x, batch_size, burn_in, n_iter) #no jit

5.63 s ± 171 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [24]:
%timeit sghmc_numba(gradU, eps, C, inv_M, theta_0, V_hat, x, batch_size, burn_in, n_iter) #jit to both

5.6 s ± 172 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [25]:
%timeit sghmc_numba2(gradU, eps, C, inv_M, theta_0, V_hat, x, batch_size, burn_in, n_iter) #jit to sghmc

5.68 s ± 155 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
